In [1]:
import torch
import torchtext
import math
import copy
import time
from typing import Tuple
from torch import Tensor
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torch.utils.data import dataset

In [2]:
from torchtext.datasets import PennTreebank 

In [3]:
train_iter = PennTreebank(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = torchtext.vocab.build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [4]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
  """Converts raw text into a flat Tensor."""
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [5]:
train_iter, val_iter, test_iter = PennTreebank()
train_data= data_process(train_iter)
val_data= data_process(val_iter)
test_data= data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [7]:
batch_size = 40
eval_batch_size = 40

In [8]:
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [9]:
print(train_data[0])

tensor([9892,  107,    4,  882,   31,    0,   16,  708, 1925,    2,  178,  301,
           3, 1887, 2295,  947,    5,    1,   42, 5643, 3412,   62, 1611,    1,
           1,   91,  730,  588, 3722,    3, 5722,    7, 2124,    3, 1678,    0,
         796, 2206,  829, 2385])


In [10]:
bptt = 40
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [11]:
class BiLSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
    super().__init__()

    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                        dropout=dropout_rate, bidirectional=True)
    self.dropout = nn.Dropout(dropout_rate)
    self.linear = nn.Linear(hidden_dim*2, vocab_size) # update hidden_dim*2

    self.init_weights()

  def forward(self, x, hidden):
    output  = self.embedding(x)
    output, hidden = self.lstm(output, hidden)
    output = self.dropout(output)
    output = self.linear(output)
    return output, hidden

  def init_weights(self):
    init_range_emb = 0.1
    init_range_other = 1/math.sqrt(self.hidden_dim)
    self.embedding.weight.data.uniform_(-init_range_emb, init_range_emb)
    self.linear.weight.data.uniform_(-init_range_other, init_range_other)
    self.linear.bias.data.zero_()


  def init_hidden(self, batch_size):
    hidden = torch.zeros(self.num_layers*2, batch_size, self.hidden_dim).to(device)
    cell = torch.zeros(self.num_layers*2, batch_size, self.hidden_dim).to(device)
    return hidden, cell
  
  def detach_hidden(self, hidden):
    hidden, cell = hidden
    hidden = hidden.detach()
    cell = cell.detach()
    return hidden, cell

In [12]:
class LSTM(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights):
    super().__init__()

    self.num_layers = num_layers
    self.hidden_dim = hidden_dim
    self.embedding_dim = embedding_dim

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=num_layers, 
                    dropout=dropout_rate, batch_first=True)
    self.dropout = nn.Dropout(dropout_rate)
    self.linear = nn.Linear(hidden_dim, vocab_size)

    if tie_weights:
      #Embedding and hidden layer need to be same size for weight tieing
          assert embedding_dim == hidden_dim, 'cannot tie, check dims'
          self.linear.weight = self.embedding.weight

  def forward(self, x):
    # x is a batch of input sequences
    x = self.embedding(x)
    x, _ = self.lstm(x)
    x = self.dropout(x)
    #next_word = x[-1, :, :]
    x = self.linear(x)
    return x 

In [13]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 100
num_layers = 2
dropout_rate = 0.4
tie_weights = True
model = LSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate, tie_weights)
model.to(device)

LSTM(
  (embedding): Embedding(9922, 100)
  (lstm): LSTM(100, 100, num_layers=2, batch_first=True, dropout=0.4)
  (dropout): Dropout(p=0.4, inplace=False)
  (linear): Linear(in_features=100, out_features=9922, bias=True)
)

In [14]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 100
num_layers = 1
#dropout_rate = 0.4
model = BiLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)
model.to(device)

c:\Users\Colum\anaconda3\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


BiLSTM(
  (embedding): Embedding(9922, 100)
  (lstm): LSTM(100, 100, dropout=0.4, bidirectional=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (linear): Linear(in_features=200, out_features=9922, bias=True)
)

In [15]:
criterion = nn.CrossEntropyLoss()
lr = 20.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()

    hidden = model.init_hidden(batch_size)
    
    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        hidden = model.detach_hidden(hidden)
        data, targets = get_batch(train_data, i)
        output, hidden = model(data, hidden)
        loss = criterion(output.view(-1, vocab_size), targets)

        optimizer.zero_grad()
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

In [16]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            hidden = model.detach_hidden(hidden)
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            output, hidden = model(data, hidden)
            output_flat = output.view(-1, vocab_size)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [17]:
best_val_loss = float('inf')
epochs = 10
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/  577 batches | lr 20.00 | ms/batch 190.59 | loss 11.23 | ppl 75355.16
| epoch   1 |   400/  577 batches | lr 20.00 | ms/batch 199.84 | loss  7.54 | ppl  1876.16
-----------------------------------------------------------------------------------------
| end of epoch   1 | time: 117.27s | valid loss  7.17 | valid ppl  1300.48
-----------------------------------------------------------------------------------------
| epoch   2 |   200/  577 batches | lr 19.00 | ms/batch 201.98 | loss  6.57 | ppl   711.14
| epoch   2 |   400/  577 batches | lr 19.00 | ms/batch 191.85 | loss  6.41 | ppl   606.84
-----------------------------------------------------------------------------------------
| end of epoch   2 | time: 117.00s | valid loss  5.79 | valid ppl   325.60
-----------------------------------------------------------------------------------------
| epoch   3 |   200/  577 batches | lr 18.05 | ms/batch 199.11 | loss  5.99 | ppl   400.87
| epoch   3 |   400/  577 batches |

In [18]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  4.67 | test ppl   107.07


In [19]:
import torch.nn.functional as F

In [20]:
def predict_next_word(
    model: nn.Module,
    prompt: str,
    vocab: torchtext.vocab.Vocab,
    top_k: int,
    temperature: float = 1.0,
) -> str:
    # Tokenize the prompt and convert to tensor
    tokenized_prompt = tokenizer(prompt)
    tensor_prompt = torch.tensor(vocab(tokenized_prompt), dtype=torch.long)
    prompt_length = len(tokenized_prompt)

    # Pass the prompt through the model
    with torch.no_grad():
        model.eval()  # turn on evaluation mode
        hidden = model.init_hidden(prompt_length)
        output, _ = model(tensor_prompt.unsqueeze(0), hidden)#.to(device)
        output = output[-1, :, :]

    # Sample the next word from the output
    output = output / temperature
    probs = F.softmax(output, dim=-1)
    top_k_probs, top_k_words = probs.topk(k=top_k)
    next_word_list = [
        (vocab.get_itos()[i[0].item()], p[0].item())
        for i, p in zip(top_k_words, top_k_probs)
    ]
    return next_word_list

In [21]:
prompt = "She doesn't study German on"
tokenized_prompt = tokenizer(prompt)
tensor_prompt = torch.tensor(vocab(tokenized_prompt), dtype=torch.long)
#prompt_length = 
print(tokenized_prompt)

['she', 'doesn', "'", 't', 'study', 'german', 'on']


In [22]:
print(len(tokenized_prompt))

7


In [23]:
LSTM = predict_next_word(model, "She doesn't study German on", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

is
in
s
in
to
had
a



In [24]:
LSTM = predict_next_word(model, "Does she live in", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

does
is
is
in



In [25]:
LSTM = predict_next_word(model, "My sister and I don’t see each other", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

to
be
and
in
in
new
some
for



In [26]:
LSTM = predict_next_word(model, "Tomorrow early morning first I'll go for a morning", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

to
had
to
bonds
in
s
in
is
for
a



In [27]:
LSTM = predict_next_word(model, "The police department in my town is just around the corner from my", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

for
to
and
in
to
for
a
to
president
for



In [28]:
LSTM = predict_next_word(model, "My mother had also wanted to make an appointment with the", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

to
and
had
also
n
a
do
average
is
for



In [29]:
LSTM = predict_next_word(model, "She doesn’t teach", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

is
in
with



In [30]:
LSTM = predict_next_word(model, "She always forgets her", vocab, top_k=5)
LSTM_Pred = ""
i = 0
for s, n in LSTM:
  if i < 10:
    i += 1
    LSTM_Pred += s + "\n"
  else:
    break

print(LSTM_Pred)

is
it
in
'



In [31]:
torch.save(best_model.state_dict(), 'C:/Users/Colum/Documents/CS4125Repo/cs4125_project/FYP/Models/BiLSTM_Model.pt')